<center> <h1 style="background-color:DarkSlateBlue; color:white" >Customer segmentation - Credit Card Clustering </h1> 

![Image](https://mainstreetlaunch.org/wp-content/uploads/2015/10/Blog-image-2015-11-11.jpg)

Customer segmentation is a way to split customers into groups based on certain characteristics that those customers have. There are different types of customer segmentation models that can be performed (geographic,demographic, behavioural...) and we can also combine them for a better understaning of our clients. The benefits of implementing such models include developing more customized marketing campaigns,attract and convert quality leads, make current customers more profitable, identify niche market opportunities and many more. 

Today we are gonna explain a common technique to perform this segmentation using clustering. The dataset for this notebook only contains variables related to the usage of credit cards, and furthermore, all the variables are numerical which will make things a bit easier in terms of modeling and model explainability. If you are interested in implementing a customer segmentation model that contains both numerical and categorical data, check this link.

A very important step we need to do before starting analysing our data, is to understand the data we are given and the meaning of each column. We are given this information [here](https://www.kaggle.com/arjunbhasin2013/ccdata), but I am going to rewrite it again since it is really important that we understand our data in order to correctly interpret the results.

- **CUST_ID**: Identifies each customer
- **BALANCE**: Amount left in the account available to make purchases
- **BALANCE_FREQUENCY**: This column gives us an idea about how frequently the balance is updated (1=frequently updated, 0=not frequently updated)
- **PURCHASES**: Amount purchased by each customer 
- **ONEOFF_PURCHASES**: Maximum purchase amount done in one-go
- **INSTALLMENTS_PURCHASES**: Amount of purchase done in installment
- **CASH_ADVANCE**: Cash in advance given by the user
- **PURCHASES_FREQUENCY**: How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
- **ONEOFF_PURCHASES_FREQUENCY**: How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
- **PURCHASES_INSTALLMENTS_FREQUENCY**: How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
- **CASH_ADVANCE_FREQUENCY**: How frequently the cash in advance being paid
- **CASH_ADVANCE_TRX**: Number of Transactions made with "Cash in Advanced"
- **PURCHASES_TRX**: Number of purchase transactions made
- **CREDIT_LIMIT**: Limit of Credit Card for user
- **PAYMENTS**: Amount of Payment done by user
- **MINIMUM_PAYMENTS**: Minimum amount of payments made by user
- **PRC_FULL_PAYMENT**: Percent of full payment paid by user
- **TENURE**: Tenure of credit card service for user


Having said all this, let's get to work!

<center>
<br>    
<a id="top"></a>    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:DarkSlateBlue; color:white" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#EDA" role="tab" aria-controls="profile" style="color:DarkSlateBlue">EDA<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">1</span></a>
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#KMeans" role="tab" aria-controls="profile" style="color:DarkSlateBlue">KMeans<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">2</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#PCA representation" role="tab" aria-controls="profile" style="color:DarkSlateBlue">PCA representation<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">3</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Cluster interpretation" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Cluster interpretation<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">4</span></a>

In [1]:
#########################################################Libraries#######################################################################
#Data analysis libraries
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",110)
pd.set_option("display.max_rows",100)

#Visualization libraries
from matplotlib import pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode (connected = True)
import ipywidgets

#Sklearn libraries
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler,RobustScaler,PowerTransformer
from sklearn.decomposition import PCA

<a id='EDA'></a>
<h1 style="color:DarkSlateBlue" >EDA</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [2]:
# Let's read the file
df = pd.read_csv("./dat/credit_card_details.csv")
print("Nuestro dataset tiene {0} columnas y {1} filas".format(df.shape[0],df.shape[1]))
display(df.head(10))
df.describe()

Nuestro dataset tiene 8950 columnas y 18 filas


,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.00,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12
5,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.000000,0,8,1800.0,1400.057770,2407.246035,0.000000,12
6,C10007,627.260806,1.000000,7091.01,6402.63,688.38,0.000000,1.000000,1.000000,1.000000,0.000000,0,64,13500.0,6354.314328,198.065894,1.000000,12
7,C10008,1823.652743,1.000000,436.20,0.00,436.20,0.000000,1.000000,0.000000,1.000000,0.000000,0,12,2300.0,679.065082,532.033990,0.000000,12
8,C10009,1014.926473,1.000000,861.49,661.49,200.00,0.000000,0.333333,0.083333,0.250000,0.000000,0,5,7000.0,688.278568,311.963409,0.000000,12
9,C10010,152.225975,0.545455,1281.60,1281.60,0.00,0.000000,0.166667,0.166667,0.000000,0.000000,0,3,11000.0,1164.770591,100.302262,0.000000,12


,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8949.000000,8950.000000,8637.000000,8950.000000,8950.000000
mean,1564.474828,0.877271,1003.204834,592.437371,411.067645,978.871112,0.490351,0.202458,0.364437,0.135144,3.248827,14.709832,4494.449450,1733.143852,864.206542,0.153715,11.517318
std,2081.531879,0.236904,2136.634782,1659.887917,904.338115,2097.163877,0.401371,0.298336,0.397448,0.200121,6.824647,24.857649,3638.815725,2895.063757,2372.446607,0.292499,1.338331
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.019163,0.000000,6.000000
25%,128.281915,0.888889,39.635000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1600.000000,383.276166,169.123707,0.000000,12.000000
50%,873.385231,1.000000,361.280000,38.000000,89.000000,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,7.000000,3000.000000,856.901546,312.343947,0.000000,12.000000
75%,2054.140036,1.000000,1110.130000,577.405000,468.637500,1113.821139,0.916667,0.300000,0.750000,0.222222,4.000000,17.000000,6500.000000,1901.134317,825.485459,0.142857,12.000000
max,19043.138560,1.000000,49039.570000,40761.250000,22500.000000,47137.211760,1.000000,1.000000,1.000000,1.500000,123.000000,358.000000,30000.000000,50721.483360,76406.207520,1.000000,12.000000


In [3]:
#All the columns but the CUST_ID are numeric, so we can make a quick exploratory analysis of their distribution
numerical_columns = df.columns.tolist()
numerical_columns.remove('CUST_ID') 
@ipywidgets.interact
def plot(col = df[numerical_columns].columns):
    #Histogram
    fig = px.histogram(df,x=df[col],title="Histograma")
    fig.update_layout(autosize=False,width=800,height=300).show()
    
    #Boxplot
    fig = px.box(df,x=df[col],title="Boxplot")
    fig.update_layout(autosize=False,width=800,height=300).show()

interactive(children=(Dropdown(description='col', options=('BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOF…

In [4]:
# We can also check if there are any correlations
corr_matrix = df.corr()
display(corr_matrix.head(3))

fig = go.Figure(data=go.Heatmap(
                   z= corr_matrix,
                   x=corr_matrix.columns,
                   y=corr_matrix.columns,
                   hoverongaps = True,
                    colorscale = 'Blues'))
fig.update_layout(title="Correlation matrix")
fig.show()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
BALANCE,1.000000,0.322412,0.181261,0.164350,0.126469,0.496692,-0.077944,0.073166,-0.063186,0.449218,0.385152,0.154338,0.531283,0.322802,0.398684,-0.318959,0.072692
BALANCE_FREQUENCY,0.322412,1.000000,0.133674,0.104323,0.124292,0.099388,0.229715,0.202415,0.176079,0.191873,0.141555,0.189626,0.095843,0.065008,0.132569,-0.095082,0.119776
PURCHASES,0.181261,0.133674,1.000000,0.916845,0.679896,-0.051474,0.393017,0.498430,0.315567,-0.120143,-0.067175,0.689561,0.356963,0.603264,0.093860,0.180379,0.086288


We can clearly see that there are **some pairs of columns that are strongly correlated**, so we will just keep one of every correlated pair.
- **PURCHASES, ONEOFF_PURCHASES and PURCHASES_TRX**: it makes sense that the amount in purchases is strongly positively correlated with the number of purchases (we will just keep PURCHASES)
- **PURCHASES_FREQUENCY AND PURCHASES_INSTALLMENTS_FREQUENCY**: we will just keep PURCHASES_FREQUENCY
- **CASH_ADVANCE_FREQUENCY AND CASH_ADVANCE_TRX** : we will just keep CASH_ADVANCE_FREQUENCY

In [5]:
model_variables = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY', 'CASH_ADVANCE_FREQUENCY', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'] #13 columns

In [6]:
#Null values (we need to impute them before performing KMeans algorithm)
display(df[model_variables].isnull().sum())

####We just need to impute the values of two columns#####
display(df[df['CREDIT_LIMIT'].isna()].head())
display(df[df['MINIMUM_PAYMENTS'].isna()].head())

#CREDIT_LIMIT:we will impute the value with 0
df['CREDIT_LIMIT'] = df['CREDIT_LIMIT'].fillna(0)

#MINIMUM_PAYMENTS:we will also impute the values with 0 
df['MINIMUM_PAYMENTS'] = df['MINIMUM_PAYMENTS'].fillna(0)

BALANCE                         0
BALANCE_FREQUENCY               0
PURCHASES                       0
INSTALLMENTS_PURCHASES          0
CASH_ADVANCE                    0
PURCHASES_FREQUENCY             0
ONEOFF_PURCHASES_FREQUENCY      0
CASH_ADVANCE_FREQUENCY          0
CREDIT_LIMIT                    1
PAYMENTS                        0
MINIMUM_PAYMENTS              313
PRC_FULL_PAYMENT                0
TENURE                          0
dtype: int64

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
5203,C15349,18.400472,0.166667,0.0,0.0,0.0,186.853063,0.0,0.0,0.0,0.166667,1,0,NaN,9.040017,14.418723,0.0,6


,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
3,C10004,1666.670542,0.636364,1499.0,1499.0,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.0,12
45,C10047,2242.311686,1.000000,437.0,97.0,340.0,184.648692,0.333333,0.083333,0.333333,0.166667,2,5,2400.0,0.000000,NaN,0.0,12
47,C10049,3910.111237,1.000000,0.0,0.0,0.0,1980.873201,0.000000,0.000000,0.000000,0.500000,7,0,4200.0,0.000000,NaN,0.0,12
54,C10056,6.660517,0.636364,310.0,0.0,310.0,0.000000,0.666667,0.000000,0.666667,0.000000,0,8,1000.0,417.016763,NaN,0.0,12
55,C10057,1311.995984,1.000000,1283.9,1283.9,0.0,0.000000,0.250000,0.250000,0.000000,0.000000,0,6,6000.0,0.000000,NaN,0.0,12


<a id='KMeans'></a>
<h1 style="color:DarkSlateBlue" >KMeans</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In order to applying KMeans correctly, we need to scale our data since otherwise some features that have low values will be neglected by the algorithm while others with larger values will matter the most. 

There are different alternatives to scale our data...

In [7]:
#Standardization
X_standardize_scaled = StandardScaler().fit_transform(df[model_variables])
df_standardize_scaled = pd.DataFrame(X_standardize_scaled, columns=df[model_variables].columns)

#Logarithm scale
df_log_scaled = df[model_variables].apply(lambda x: np.log10(x+1))

#Power scale
X_power_scaled = PowerTransformer().fit_transform(df[model_variables])
df_power_scaled = pd.DataFrame(X_power_scaled, columns=df[model_variables].columns)

In [8]:
def evaluate_metrics(df, min_clust=2, max_clust=15, rand_state=41):
    inertia_list = []
    silhouette_list = []
    ch_score_list = []
    db_score_list = []
    for n_clust in range(min_clust, max_clust+1):
        #KMeans algorithm
        kmeans = KMeans(n_clusters=n_clust, init='k-means++',random_state=rand_state)
        y_label = kmeans.fit_predict(df)
        #Inertia
        inertia_list.append(kmeans.inertia_)
        #Silhouette Score
        silhouette_list.append(silhouette_score(df, y_label))
        #Calinski Harabasz Score
        #ch_score_list.append(calinski_harabasz_score(df, y_label))
        #Davies Bouldin Score
        #db_score_list.append(davies_bouldin_score(df, y_label))
    return inertia_list,silhouette_list

In [9]:
#Let´s calculate the inertia and silhouette coefficient for both kinds of scaling
inertia_list_st,silhouette_list_st = evaluate_metrics(df_standardize_scaled)
inertia_list_log,silhouette_list_log = evaluate_metrics(df_log_scaled)
inertia_list_power,silhouette_list_power = evaluate_metrics(df_power_scaled)

In [10]:
fig = make_subplots(rows=3, cols=2,subplot_titles=("Intertia (standardize data)","Silhouette score (standardize data)",
                                                  "Inertia (log scaled data)", "Silhouette score (log scaled data)",
                                                  "Inertia (power scaled data)", "Silhouette score (power scaled data)"))

trace1 = go.Scatter(x=list(range(2,16)),y=np.array(inertia_list_st))
trace2 = go.Scatter(x=list(range(2,16)),y=np.array(silhouette_list_st))

trace3 = go.Scatter(x=list(range(2,16)),y=np.array(inertia_list_log))
trace4 = go.Scatter(x=list(range(2,16)),y=np.array(silhouette_list_log))

trace5 = go.Scatter(x=list(range(2,16)),y=np.array(inertia_list_power))
trace6 = go.Scatter(x=list(range(2,16)),y=np.array(silhouette_list_power))


fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 3, 1)
fig.append_trace(trace6, 3, 2)

fig.update_layout(title_text="Metrics to choose the optimal number of clusters",
                  autosize=False,width=800,height=800,
                 showlegend=False)

fig.update_yaxes(range=[0.1, 0.5], row=1, col=2)
fig.update_yaxes(range=[0.1, 0.5], row=2, col=2)
fig.update_yaxes(range=[0.1, 0.5], row=3, col=2)
fig.show()

In [11]:
# Kmeans on just standardized features
kmeans_log_scaled = KMeans(n_clusters=5, n_init=100, max_iter=400, init='k-means++', random_state=41).fit(df_log_scaled)

#Labels
labels_log_scaled = kmeans_log_scaled.labels_

#Concatenation of scaled dataframe with their corresponding clustering labels
clusters_log_scaled = pd.concat([df_log_scaled, pd.DataFrame({'cluster_label':labels_log_scaled})], axis=1)

#Reformat the column cluster_label as string type
clusters_log_scaled['cluster_label'] = clusters_log_scaled['cluster_label'].astype(str)
clusters_log_scaled.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,CASH_ADVANCE_FREQUENCY,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,cluster_label
0,1.622222,0.259637,1.984077,1.984077,0.000000,0.066947,0.000000,0.000000,3.000434,2.307072,2.147707,0.00000,1.113943,3
1,3.505620,0.280827,0.000000,0.000000,3.809152,0.000000,0.000000,0.096910,3.845160,3.613211,3.030737,0.08715,1.113943,0
2,3.397270,0.301030,2.888836,0.000000,0.000000,0.301030,0.301030,0.000000,3.875119,2.794535,2.798157,0.00000,1.113943,1
3,3.222110,0.213880,3.176091,0.000000,2.315525,0.034762,0.034762,0.034762,3.875119,0.000000,0.000000,0.00000,1.113943,1
4,2.913132,0.301030,1.230449,0.000000,0.000000,0.034762,0.034762,0.000000,3.079543,2.832084,2.390566,0.00000,1.113943,1


<a id='PCA representation'></a>
<h1 style="color:DarkSlateBlue" >PCA representation</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

We will just use PCA to represent our data in a 2 dimensional space. 

In [12]:
pca = PCA().fit(df_log_scaled - df_log_scaled.mean())
print(pca.singular_values_)

pca = PCA().fit(df_log_scaled)
print(pca.singular_values_)

#Singular values
U,s,V = np.linalg.svd(df_log_scaled - df_log_scaled.mean())
print(s)

[190.60299531 129.99752806  81.43388223  65.31910811  51.19085211
  34.0503915   27.38653993   7.75041592   6.55050623   5.14262745
   4.7211006    3.7304654    3.42255294]
[190.60299531 129.99752806  81.43388223  65.31910811  51.19085211
  34.0503915   27.38653993   7.75041592   6.55050623   5.14262745
   4.7211006    3.7304654    3.42255294]
[190.60299531 129.99752806  81.43388223  65.31910811  51.19085211
  34.0503915   27.38653993   7.75041592   6.55050623   5.14262745
   4.7211006    3.7304654    3.42255294]


In [13]:
####################################################################################
#First way:let's calculate the eigen values and singular values of the covariance matrix through numpy library
print("#"*100)
cov_mat = np.cov(df_log_scaled.T)

#Eigen values of the covariance matrix
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
eigen_vals = sorted(eigen_vals,reverse=True)

#Singular values of the covariance matrix
U,s,V = np.linalg.svd(cov_mat)
sing_vals = sorted(s,reverse=True)

print("Singular values of the covariance matrix: \n", sing_vals)  # print the Eigenvalues
print("Eigenvalues of the covariance matrix: \n", eigen_vals)  # print the Eigenvalues

print("Percentage of Variance Explained by Each Component: \n", eigen_vals/sum(eigen_vals)) # the sum of the Eigenvalues 
print("The eigen values and singular values of the covariance matrix are the same. This will always be the case for positive definite symmetric matrices.")
print("#"*100)

######################################################################################
#Second way:let's calculate the singular values of the covariance matrix through sklearn library PCA
covar_matrix = PCA(n_components=13).fit(df_log_scaled)
variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
singular_values = covar_matrix.singular_values_**2/(df_log_scaled.shape[0]-1)
eigen_values = singular_values
print("Singular values of the covariance matrix:",singular_values)
print("Eigen values of the covariance matrix:",eigen_values)
variance = eigen_values/sum(eigen_values)
print("Percentage of Variance Explained by Each Component: \n", variance)
var = np.cumsum(np.round(variance, 3)*100) #cumulative sum of the variance of the principal components
print("#"*100)

##########################################################################################
#Plot of cumulative sum of variance of principal components
fig1 = px.line(x=range(1,14),y=var,title="PCA analysis",labels={'y':'% Variance Explained', 'x':'# of Features'})
#fig1.update_traces(mode='markers+lines')
fig2 = px.line(x=range(1,14), y=np.array(13 * [var[2]]),line_dash_sequence=['dash'],color_discrete_sequence = ["red"])
fig1.add_trace(fig2.data[0]).update_layout(autosize=False,width=800,height=400).show()


#######################################################################################
#Third way: we can get the eigen values of the covariance matrix from the singular values of the original matrix(once centered)
U, s, V = np.linalg.svd(df_log_scaled - df_log_scaled.mean(),full_matrices=False)
print('Singular values of the original matrix(centered): ',s)

eigen_values = s**2/(df_log_scaled.shape[0]-1)
print("Eigen values of the covariance matrix:",eigen_values)
print("The singular values of the original matrix(centered) are related to the eigen values of the covariance matrix.")



####################################################################################################
Singular values of the covariance matrix: 
 [4.05961580290195, 1.8884073417958296, 0.7410299670287755, 0.47676677663740774, 0.292826387325221, 0.12955963365369216, 0.08381076873343654, 0.0067123641600627185, 0.004794852146115108, 0.0029552594766442766, 0.0024906459755380467, 0.0015550756657537626, 0.0013089583864519963]
Eigenvalues of the covariance matrix: 
 [4.059615802901955, 1.888407341795829, 0.7410299670287743, 0.47676677663740913, 0.29282638732522065, 0.1295596336536922, 0.08381076873343665, 0.006712364160062725, 0.0047948521461151205, 0.0029552594766442757, 0.0024906459755380922, 0.0015550756657537962, 0.0013089583864520156]
Percentage of Variance Explained by Each Component: 
 [5.27782567e-01 2.45508078e-01 9.63398304e-02 6.19834992e-02
 3.80697755e-02 1.68437900e-02 1.08960712e-02 8.72661098e-04
 6.23369180e-04 3.84207400e-04 3.23803924e-04 2.02172291e-04
 1.70175073e-04]
The 

Singular values of the original matrix(centered):  [190.60299531 129.99752806  81.43388223  65.31910811  51.19085211
  34.0503915   27.38653993   7.75041592   6.55050623   5.14262745
   4.7211006    3.7304654    3.42255294]
Eigen values of the covariance matrix: [4.05961580e+00 1.88840734e+00 7.41029967e-01 4.76766777e-01
 2.92826387e-01 1.29559634e-01 8.38107687e-02 6.71236416e-03
 4.79485215e-03 2.95525948e-03 2.49064598e-03 1.55507567e-03
 1.30895839e-03]
The singular values of the original matrix(centered) are related to the eigen values of the covariance matrix.


In [14]:
# Plotting KMeans clusters on just standardized data
pca = PCA(n_components=3).fit(df_log_scaled).transform(df_log_scaled)
df_pca = pd.DataFrame(pca).rename(columns={0:'PC1',1:'PC2',2:'PC3'})
df_pca_labeled = pd.concat([df_pca, clusters_log_scaled[['cluster_label']]], axis=1)

#Figure
fig = px.scatter(df_pca_labeled.sort_values(by="cluster_label"),x="PC1",y="PC2",color="cluster_label",opacity=0.8).update_layout(autosize=False,width=800,height=400)
fig.show()

In [15]:
fig = px.scatter_3d(df_pca_labeled.sort_values(by="cluster_label"), x='PC1', y='PC2', z='PC3',color='cluster_label',height = 1000,width = 1000,title="Clustering representation with 3 main PCA components")
fig.update_traces(mode="markers",marker_symbol='circle',marker_size=10,marker_line = dict(color = 'gray',width = 5)).show()

<a id='Cluster interpretation'></a>
<h1 style="color:DarkSlateBlue" >Cluster interpretation</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [16]:
df_index = df.reset_index().reset_index().drop(columns=['level_0','index']).reset_index()
cluster_log_scaled_profile = df_index.merge(clusters_log_scaled[['cluster_label']].reset_index()).drop(columns=['index'])
cluster_log_scaled_profile['cluster_label'] = 'cluster_' + cluster_log_scaled_profile['cluster_label']
cluster_log_scaled_profile.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,cluster_label
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12,cluster_3
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12,cluster_0
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12,cluster_1
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,0.000000,0.000000,12,cluster_1
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12,cluster_1


In [17]:
fig = px.histogram(cluster_log_scaled_profile,x="cluster_label",title="Clustering distribution of our clients")
fig.update_layout(width=800,height=400).show()

In [20]:
display(cluster_log_scaled_profile.groupby("cluster_label").mean())#.sort_values(by="BALANCE",ascending=False))
@ipywidgets.interact
def plot(col = cluster_log_scaled_profile.select_dtypes(include=['float64','int']).columns):
    #category_list = cluster_log_scaled_profile.groupby("cluster_label")[col].median().reset_index().sort_values(by=col,ascending=False)['cluster_label'].tolist()
    category_orders = {'cluster_label': ['cluster_0','cluster_1','cluster_2','cluster_3','cluster_4']}
    #Boxplot
    fig = px.histogram(cluster_log_scaled_profile,
                       x= cluster_log_scaled_profile[col],
                       color = cluster_log_scaled_profile['cluster_label'],
                       color_discrete_map={"cluster_0": "red","cluster_1": "green","cluster_2": "blue","cluster_3": "goldenrod","cluster_4": "magenta"},
                       #color_discrete_sequence=px.colors.qualitative.Dark24,
                       category_orders = category_orders,
                       facet_col="cluster_label",
                       title="Histogram of column distribution. Ordered by the mean of the distribution")
    fig.update_layout(autosize=False,width=1400,height=500).show()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
cluster_label,,,,,,,,,,,,,,,,,
cluster_0,2117.446195,0.887205,13.980625,13.956689,0.075644,1951.512336,0.017808,0.017029,0.000779,0.275458,6.343816,0.252444,3992.061714,1618.519701,935.248371,0.042972,11.323417
cluster_1,783.760798,0.797571,866.664907,866.367318,0.618130,14.459621,0.360315,0.354759,0.006331,0.010606,0.159052,7.920474,4319.822848,1206.407068,479.496016,0.135367,11.564298
cluster_2,2875.931230,0.964704,1423.139715,872.742991,550.572909,2275.966941,0.660874,0.295883,0.487033,0.298321,7.626235,21.524149,5484.995121,2715.438250,1329.032970,0.060975,11.494512
cluster_3,63.999579,0.686607,348.659126,66.183836,283.047254,3.105470,0.650132,0.051188,0.585042,0.002482,0.032145,9.792453,3159.245707,411.123928,113.765004,0.340256,11.341020
cluster_4,1277.411401,0.962547,2233.809972,1184.357729,1049.936327,0.296468,0.826268,0.342392,0.706674,0.000813,0.009713,31.668363,5183.596020,2193.080774,977.082518,0.238957,11.838575


interactive(children=(Dropdown(description='col', options=('BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOF…

Cluster 3: Customers with very high usage of credit card that use it for both purchases and cash advances.

Cluster 1: Customers with very high usage of credit card that use it only for purchases and never for cash advances.

Cluster 0: Customers with high usage of credit card that use it only for cash advances and never for purchases.

Cluster 2 : Customers with medium usage of credit card that use it only for purchases and never for cash advances.

Cluster 4 : Customers with the least usage of credit card that use it only for purchases and never for cash advances.